In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keplergl import KeplerGl
from datetime import datetime
from shapely.geometry import LineString, shape, Point
from shapely import to_geojson
import zipfile
import os
import osmnx as ox
from geopy.geocoders import Nominatim
from typing import List
import h3
import ast
import altair as alt

%matplotlib inline

In [2]:
def day_load(path_way,path_header):
    
    df = pd.read_csv(path_way,compression='gzip',header=None,on_bad_lines='skip')
    headers = pd.read_csv(path_header)
    df.drop([11], axis=1, inplace=True)
    df.columns = headers.columns

    return df

In [3]:
def column_types(df):
    df['CaptureDate'] = pd.to_datetime(df['CaptureDate'], format='%Y-%m-%dT%H:%M:%S.%fZ')#, utc=True)
    df['RawSpeed'] = df['RawSpeed'].astype(float)
    return df

In [4]:
def polygon_check(lon, lat, geom):
    polygon = shape(geom['geometry'])
    point = Point(lon, lat)
    return polygon.contains(point)

In [5]:
#date_filter = list of dow_dict kexs, eg. ['Montag', 'Dienstag']
#date format:  pd.datetime

# Weekday-codes
dow_dict = {'Montag':0, 'Dienstag':1, 'Mittwoch':2, 'Donnerstag':3, 'Freitag':4, 'Samstag':5, 'Sonntag':6}

def filter_days(dates,date_filter):
    days = []
    date_filter = [dow_dict[x] for x in date_filter]

    for date in dates:
        dow = date.weekday()
        if dow in date_filter:
            date_filter.append(date)
        else:
            pass

In [6]:
# start, end = int of full hours, eg. 0,1,2 ...abs

def filter_hours(df,start,end):
    df['hour'] = df['timestamp'].apply(lambda x: x.hour)
    df = df[(df['hour'] >= start) & (df['hour']<end)]
    df = df.drop(['hour'],axis=1)
    df.reset_index(drop=True,inplace=True)
    return df

In [7]:
#geom is ecpected to be geojson format
### take care to take that one level of the nested json --> ['features'][0]

def shape_filter(df, geom, bbox:bool=False):
    lng_min, lng_max, lat_min, lat_max = 100, 0, 100, 0
    geom_shape = geom['geometry']['coordinates'][0]
    for i in geom_shape:
        if i[0] < lng_min:
            lng_min = i[0]
        elif i[0] > lng_max:
            lng_max = i[0]
        else:
            pass
        if i[1] < lat_min:
            lat_min = i[1]
        elif i[1] > lat_max:
            lat_max = i[1]
        else:
            pass

    sub_df = df[df['Longitude'] > lng_min]
    sub_df = sub_df[sub_df['Longitude'] < lng_max]
    sub_df = sub_df[sub_df['Latitude'] > lat_min]
    sub_df = sub_df[sub_df['Latitude'] < lat_max]

    if bbox==False:
        sub_df['geo_check'] = sub_df.apply(lambda x: polygon_check(x['Longitude'], x['Latitude'],geom), axis=1)
        sub_df = sub_df[sub_df['geo_check']].drop('geo_check',axis=1)
    sub_df.reset_index(drop=True,inplace=True)
    return sub_df

In [8]:
def create_dicts(G):
    nodes = ox.graph_to_gdfs(G, edges=False)
    nodes_dict = nodes.to_dict()

    G_edges = list(G.edges())
    G_shapes = []
    G_names = []
    for edge in G_edges:
        try:
            name_i = G.get_edge_data(edge[0],edge[1])[0]['name']
            G_names.append(name_i)
        except:
            G_names.append(None)
        try:
            shape_i = G.get_edge_data(edge[0],edge[1])[0]['geometry']
            G_shapes.append(shape_i)
        except:
            start = (nodes_dict['x'][edge[0]],nodes_dict['y'][edge[0]])
            end = (nodes_dict['x'][edge[1]],nodes_dict['y'][edge[1]])
            G_shapes.append(LineString([start,end]))

    #G_edges = [str(edge) for edge in G_edges]
    #G_shapes = [str(edge) for edge in G_shapes]
    #G_names = [str(edge) for edge in G_names]


    edge_name_dict = dict(zip(G_edges,G_names))
    edge_shape_dict = dict(zip(G_edges,G_shapes))
    return edge_name_dict, edge_shape_dict

In [9]:
def haversine_distance_to_point(lat1:float, lon1:float, lat2:List[float], lon2:List[float]) -> List[float]:
    """_summary_

    Args:
        lat1 (float): latitude of poi
        lon1 (float): longitude of poi
        lat2 (List[float]): latitudes of query points
        lon2 (List[float]): longitudes of query points

    Returns:
        List[float]: distances from each query point to the provided poi
    """
    # Radius of the Earth in kilometers
    R = 6378.137
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = np.radians(lat1), np.radians(lon1), np.radians(lat2), np.radians(lon2)
    
    # Compute the differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    
    return distance

In [10]:
# filters street based on name either it is stored equal to street name or nested in list with street names

def street_filter(df,street):
    df_street = df[df.applymap(lambda x: street in x if isinstance(x, list) else x == street).any(axis=1)]
    trip_ids = df_street['TripId'].unique()
    return trip_ids

In [11]:
df_list = []

path = '../data/external/metzingen/waypoints/'
headers = pd.read_csv('../data/external/metzingen/TripBulkReportWaypointsHeaders.csv')

for file in os.listdir(path):
    file_path = os.path.join(path, file)
    df = pd.read_csv(file_path,header=None,on_bad_lines='skip') # compression='gzip'
    df.drop([11], axis=1, inplace=True)
    df_list.append(df)
    
df = pd.concat(df_list)
df.columns = headers.columns

In [12]:
metzingen={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              9.26143167487868,
              48.55003593215798
            ],
            [
              9.26143167487868,
              48.527762954288534
            ],
            [
              9.30179387158708,
              48.527762954288534
            ],
            [
              9.30179387158708,
              48.55003593215798
            ],
            [
              9.26143167487868,
              48.55003593215798
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

metzingen = metzingen['features'][0]

In [13]:
#df['CaptureDate']

0           2024-05-10T03:31:05.999Z
1           2024-05-10T03:31:08.999Z
2           2024-05-10T03:31:11.999Z
3           2024-05-10T03:31:14.999Z
4           2024-05-10T03:31:17.999Z
                      ...           
10587687    2024-05-10T03:30:53.999Z
10587688    2024-05-10T03:30:56.999Z
10587689    2024-05-10T03:30:59.999Z
10587690    2024-05-10T03:31:01.999Z
10587691    2024-05-10T03:31:02.999Z
Name: CaptureDate, Length: 45131346, dtype: object

In [ ]:
"""df_sorted = df.sort_values(by='CaptureDate')
first_time_point = df_sorted.CaptureDate.iloc[0]
last_time_point = df_sorted.CaptureDate.iloc[-1]"""

In [15]:
df = shape_filter(df, metzingen)

In [16]:
len(df)

3061318

In [17]:
edge_lookup = pd.read_csv('../data/interim/metzingen/lookup_metzingen.csv')
edge_lookup['edges'] = edge_lookup['edges'].apply(lambda x: ast.literal_eval(x))
edge_lookup.head(2)

,h3,edges,distances
0,8d1faa68e1618bf,"(251988386, 76700458)",0.000382
1,8d1faa68e16183f,"(251988386, 76700458)",0.000330


In [18]:
G = ox.graph_from_place('Metzingen, Germany', network_type='drive')

edge_name_dict, edge_shape_dict = create_dicts(G)

In [18]:
"""edge_lookup['edge_name'] = edge_lookup['edges'].map(edge_name_dict)
edge_lookup"""

"edge_lookup['edge_name'] = edge_lookup['edges'].map(edge_name_dict)\nedge_lookup"

In [19]:
aperture_size = 13
batch_size = 10000 

num_batches = len(df) // batch_size + 1

processed_chunks = []

for i in range(num_batches):
    batch = df.iloc[i*batch_size:(i+1)*batch_size]
    batch['h3'] = batch.apply(lambda x: h3.geo_to_h3(x['Latitude'], x['Longitude'], aperture_size), axis=1)
    processed_chunks.append(batch)

df = pd.concat(processed_chunks, ignore_index=True)

C:\Users\lukas\AppData\Local\Temp\ipykernel_24792\1424820356.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['h3'] = batch.apply(lambda x: h3.geo_to_h3(x['Latitude'], x['Longitude'], aperture_size), axis=1)
C:\Users\lukas\AppData\Local\Temp\ipykernel_24792\1424820356.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['h3'] = batch.apply(lambda x: h3.geo_to_h3(x['Latitude'], x['Longitude'], aperture_size), axis=1)
C:\Users\lukas\AppData\Local\Temp\ipykernel_24792\1424820356.py:10: SettingW

In [20]:
df.head(2)

,TripId,WaypointSequence,CaptureDate,Latitude,Longitude,SegmentId,ZoneName,Frc,DeviceId,RawSpeed,RawSpeedMetric,h3
0,cc74e58360af3cf0a62684ba3869056e,319,2024-05-10T03:42:46.000Z,48.528002,9.262295,NaN,NaN,NaN,8a39f01833b609e879893d5d0647c85e,86.4,kph,8d1f8cd345616ff
1,cc74e58360af3cf0a62684ba3869056e,320,2024-05-10T03:42:49.000Z,48.528581,9.262758,NaN,NaN,NaN,8a39f01833b609e879893d5d0647c85e,82.8,kph,8d1f8cd3419237f


In [21]:
df_merged = pd.merge(df,edge_lookup,on='h3',how='inner')

In [22]:
df_merged.head(2)

,TripId,WaypointSequence,CaptureDate,Latitude,Longitude,SegmentId,ZoneName,Frc,DeviceId,RawSpeed,RawSpeedMetric,h3,edges,distances
0,cc74e58360af3cf0a62684ba3869056e,319,2024-05-10T03:42:46.000Z,48.528002,9.262295,NaN,NaN,NaN,8a39f01833b609e879893d5d0647c85e,86.4,kph,8d1f8cd345616ff,"(403507338, 1508756532)",0.000039
1,cc74e58360af3cf0a62684ba3869056e,320,2024-05-10T03:42:49.000Z,48.528581,9.262758,NaN,NaN,NaN,8a39f01833b609e879893d5d0647c85e,82.8,kph,8d1f8cd3419237f,"(403507338, 1508756532)",0.000006


In [23]:
len(df_merged)

3026248

In [29]:
df_merged.head(3)
df = df_merged

In [30]:
df['shape'] = df['edges'].map(edge_shape_dict)

In [32]:
df['shape'] = df['edges'].map(edge_shape_dict)

,TripId,WaypointSequence,CaptureDate,Latitude,Longitude,SegmentId,ZoneName,Frc,DeviceId,RawSpeed,RawSpeedMetric,h3,edges,distances,shape
0,cc74e58360af3cf0a62684ba3869056e,319,2024-05-10T03:42:46.000Z,48.528002,9.262295,NaN,NaN,NaN,8a39f01833b609e879893d5d0647c85e,86.4,kph,8d1f8cd345616ff,"(403507338, 1508756532)",0.000039,"LINESTRING (9.2597405 48.5262529, 9.2600527 48..."
1,cc74e58360af3cf0a62684ba3869056e,320,2024-05-10T03:42:49.000Z,48.528581,9.262758,NaN,NaN,NaN,8a39f01833b609e879893d5d0647c85e,82.8,kph,8d1f8cd3419237f,"(403507338, 1508756532)",0.000006,"LINESTRING (9.2597405 48.5262529, 9.2600527 48..."
2,cc74e58360af3cf0a62684ba3869056e,321,2024-05-10T03:42:52.000Z,48.529173,9.263110,NaN,NaN,NaN,8a39f01833b609e879893d5d0647c85e,79.2,kph,8d1f8cd341956ff,"(1508756532, 1508756550)",0.000062,"LINESTRING (9.2630486 48.5291162, 9.2632318 48..."


In [33]:
df_merged['street_name'] = df_merged['edges'].map(edge_name_dict)

In [34]:
len(df[df.street_name=='Lindenplatz'])

8588

In [35]:
street = 'Lindenplatz'

In [36]:
def street_filter(df,street):
    df_street = df[df.applymap(lambda x: street in x if isinstance(x, list) else x == street).any(axis=1)]
    trip_ids = df_street['TripId'].unique()
    return trip_ids

In [135]:
street = 'Lindenplatz'

In [137]:
df[df.applymap(lambda x: street in x if isinstance(x, list) else x == street).any(axis=1)].head(3)

C:\Users\lukas\AppData\Local\Temp\ipykernel_28924\1523254629.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[df.applymap(lambda x: street in x if isinstance(x, list) else x == street).any(axis=1)].head(3)


,TripId,WaypointSequence,CaptureDate,Latitude,Longitude,SegmentId,ZoneName,Frc,DeviceId,RawSpeed,RawSpeedMetric,h3,edges,distances,street_name,shape
337,cc808266f6130c25816ba0226676f476,818,2024-05-07T09:01:35.997Z,48.537447,9.282486,NaN,NaN,NaN,80ac874e379ad416dcf0d1ed1e3c578c,36.0,kph,8d1f8cd34a6e47f,"(26558861, 299324119)",0.000006,Lindenplatz,"LINESTRING (9.2823316 48.5376082, 9.2823776 48..."
338,cc808266f6130c25816ba0226676f476,819,2024-05-07T09:01:38.997Z,48.537226,9.282821,NaN,NaN,NaN,80ac874e379ad416dcf0d1ed1e3c578c,39.6,kph,8d1f8cd34a6acff,"(299324119, 26558861)",0.000032,Lindenplatz,"LINESTRING (9.2831346 48.5371026, 9.2829731 48..."
339,cc808266f6130c25816ba0226676f476,820,2024-05-07T09:01:41.997Z,48.537056,9.283208,NaN,NaN,NaN,80ac874e379ad416dcf0d1ed1e3c578c,39.6,kph,8d1f8cd34a4c9bf,"(2250005494, 299324119)",0.000021,"[Lindenplatz, Ulmer Straße]","LINESTRING (9.2840436 48.536897, 9.2839908 48...."


In [101]:
#df[df.street_name == 'Lindenplatz']

In [116]:
len(df['TripId'])

3026248

In [105]:
df_filtered = df[df['TripId'].isin(trips_i)].reset_index(drop=True)

In [113]:
df_filtered.street_name.notna().sum()/len(df_filtered)

np.float64(0.8757283522093599)

In [53]:
df_filtered.drop('shape', axis=1,inplace=True)

In [56]:
df.drop('shape', inplace=True, axis=1)

In [58]:
map1 = KeplerGl(data = {'filtered': df_filtered, 'unfiltered':df})
map1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


C:\Users\lukas\AppData\Roaming\Python\Python312\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(data={'filtered':                                  TripId  WaypointSequence  \
0      ffdccc03c0b79cc…

In [ ]:
################ PARKSUCHVERKEHR

In [ ]:
################################ @FABI kannst du uns hier Parksuchverkehr ausgeben?

In [243]:
df_list = []

path = '../data/external/metzingen/waypoints/'
headers = pd.read_csv('../data/external/metzingen/TripBulkReportWaypointsHeaders.csv')

for file in os.listdir(path):
    file_path = os.path.join(path, file)
    df = pd.read_csv(file_path,header=None,on_bad_lines='skip') # compression='gzip'
    df.drop([11], axis=1, inplace=True)
    df_list.append(df)
    
df = pd.concat(df_list)
df.columns = headers.columns

In [227]:
trip_df = pd.read_csv('../Data/external/metzingen/trips.csv',header=None,on_bad_lines='skip')

headers = pd.read_csv('../data/external/metzingen/TripBulkReportTripsHeaders.csv')
# what is this?
trip_df.drop([31], axis=1, inplace=True)
trip_df.columns = headers.columns

trip_df['h3'] = trip_df.apply(lambda x: h3.geo_to_h3(x.EndLocLat,x.EndLocLon,13),1)

In [245]:
df_merged = pd.merge(trip_df,edge_lookup,on='h3',how='inner')

df_merged['street_name'] = df_merged['edges'].map(edge_name_dict)

In [246]:
trip_df = street_filter(df_merged,'Wilhelmstraße').tolist()

C:\Users\lukas\AppData\Local\Temp\ipykernel_28924\3314790284.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_street = df[df.applymap(lambda x: street in x if isinstance(x, list) else x == street).any(axis=1)]


In [249]:
df[df['TripId'].isin(parking_trips)]

,TripId,WaypointSequence,CaptureDate,Latitude,Longitude,SegmentId,ZoneName,Frc,DeviceId,RawSpeed,RawSpeedMetric
144200,ce0049c168aa0d96083e9c5ea4b1e650,0,2024-05-12T12:38:42.998Z,50.668270,12.608750,NaN,NaN,NaN,9a061e2fbbb1c2de1595f0d3afe909a6,32.4,kph
144201,ce0049c168aa0d96083e9c5ea4b1e650,1,2024-05-12T12:38:45.997Z,50.668336,12.608245,NaN,NaN,NaN,9a061e2fbbb1c2de1595f0d3afe909a6,43.2,kph
144202,ce0049c168aa0d96083e9c5ea4b1e650,2,2024-05-12T12:38:48.997Z,50.668415,12.607717,NaN,NaN,NaN,9a061e2fbbb1c2de1595f0d3afe909a6,43.2,kph
144203,ce0049c168aa0d96083e9c5ea4b1e650,3,2024-05-12T12:38:51.997Z,50.668508,12.607148,NaN,NaN,NaN,9a061e2fbbb1c2de1595f0d3afe909a6,50.4,kph
144204,ce0049c168aa0d96083e9c5ea4b1e650,4,2024-05-12T12:38:54.997Z,50.668599,12.606561,NaN,NaN,NaN,9a061e2fbbb1c2de1595f0d3afe909a6,50.4,kph
...,...,...,...,...,...,...,...,...,...,...,...
10427683,cab818e38861cc234f4ffbd97630c020,905,2024-05-10T07:06:28.999Z,48.537684,9.282089,NaN,NaN,NaN,d56e15b21af73c54fdebfc458337be19,21.6,kph
10427684,cab818e38861cc234f4ffbd97630c020,906,2024-05-10T07:06:31.999Z,48.537525,9.281848,NaN,NaN,NaN,d56e15b21af73c54fdebfc458337be19,25.2,kph
10427685,cab818e38861cc234f4ffbd97630c020,907,2024-05-10T07:06:34.999Z,48.537313,9.281669,NaN,NaN,NaN,d56e15b21af73c54fdebfc458337be19,32.4,kph
10427686,cab818e38861cc234f4ffbd97630c020,908,2024-05-10T07:06:35.999Z,48.537236,9.281631,NaN,NaN,NaN,d56e15b21af73c54fdebfc458337be19,32.4,kph


In [250]:
df = column_types(df)

In [251]:
parking_df = pd.DataFrame([])
distances = []

for trip in parking_trips:
    trip_i = df[df['TripId']==trip]
    end_time = trip_i.loc[:,'CaptureDate'].max()
    trip_i['timedelta'] = end_time - trip_i.loc[:,'CaptureDate']
    trip_i.loc[:,'timedelta'] = trip_i.loc[:,'timedelta'].apply(lambda x: x.total_seconds()/60)
    parking_i = trip_i[trip_i['timedelta']<5]
    lng_end = parking_i[parking_i['timedelta']==parking_i['timedelta'].min()].reset_index()['Longitude'][0]
    lat_end = parking_i[parking_i['timedelta']==parking_i['timedelta'].min()].reset_index()['Latitude'][0]

    parking_i['distance'] = haversine_distance_to_point(lat_end,lng_end,parking_i['Latitude'],parking_i['Longitude'])
    distances.append(parking_i[parking_i['timedelta']==parking_i['timedelta'].max()].reset_index()['distance'][0])
    parking_df = pd.concat([parking_df,parking_i],axis=0,ignore_index=True)

C:\Users\lukas\AppData\Local\Temp\ipykernel_28924\1624382306.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip_i['timedelta'] = end_time - trip_i.loc[:,'CaptureDate']
C:\Users\lukas\AppData\Local\Temp\ipykernel_28924\1624382306.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[5.695e+01 5.690e+01 5.685e+01 ... 1.000e-01 5.000e-02 0.000e+00]' has dtype incompatible with timedelta64[ns], please explicitly cast to a compatible dtype first.
  trip_i.loc[:,'timedelta'] = trip_i.loc[:,'timedelta'].apply(lambda x: x.total_seconds()/60)
C:\Users\lukas\AppData\Local\Temp\ipykernel_28924\1624382306.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

KeyboardInterrupt: 